In [ ]:
# =============================
# Face Mask Detection with MobileNetV2
# =============================

# Install required libraries (if not already installed)
# !pip install tensorflow opencv-python matplotlib

import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
import cv2

# =============================
# PARAMETERS
# =============================
INIT_LR = 1e-4
EPOCHS = 20
BS = 32
IMAGE_SIZE = 224

# =============================
# DATASET LOADING
# =============================
# Dataset structure should be like:
# dataset/
# ├── with_mask/
# └── without_mask/

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest",
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    "dataset",
    subset="training",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BS,
    class_mode="binary"
)

val_generator = train_datagen.flow_from_directory(
    "dataset",
    subset="validation",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BS,
    class_mode="binary"
)

# =============================
# BUILD MODEL (MobileNetV2)
# =============================
baseModel = MobileNetV2(weights="imagenet", include_top=False,
                        input_tensor=Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(1, activation="sigmoid")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)

# Freeze base model layers
for layer in baseModel.layers:
    layer.trainable = False

# =============================
# COMPILE MODEL
# =============================
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
              metrics=["accuracy"])

# =============================
# TRAIN MODEL
# =============================
print("[INFO] training head...")
H = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator) // BS,
    validation_data=val_generator,
    validation_steps=len(val_generator) // BS,
    epochs=EPOCHS
)

# =============================
# SAVE MODEL
# =============================
model.save("mask_detector.model", save_format="h5")

# =============================
# EVALUATION
# =============================
print("[INFO] evaluating network...")
val_generator.reset()
predIdxs = model.predict(val_generator, steps=(len(val_generator) // BS) + 1)
predIdxs = np.where(predIdxs > 0.5, 1, 0)

print(classification_report(val_generator.classes, predIdxs,
                            target_names=["with_mask", "without_mask"]))

# =============================
# PLOT TRAINING
# =============================
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, EPOCHS), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, EPOCHS), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, EPOCHS), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, EPOCHS), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")
plt.show()

# =============================
# REAL-TIME DETECTION (Webcam)
# =============================
def detect_and_predict_mask(frame, faceNet, maskNet):
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),
                                 (104.0, 177.0, 123.0))

    faceNet.setInput(blob)
    detections = faceNet.forward()
    faces = []
    locs = []
    preds = []

    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (IMAGE_SIZE, IMAGE_SIZE))
            face = tf.keras.preprocessing.image.img_to_array(face)
            face = np.expand_dims(face, axis=0) / 255.0

            faces.append(face)
            locs.append((startX, startY, endX, endY))

    if len(faces) > 0:
        preds = maskNet.predict(faces)

    return (locs, preds)

# Load OpenCV DNN face detector
prototxtPath = "face_detector/deploy.prototxt"
weightsPath = "face_detector/res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

maskNet = tf.keras.models.load_model("mask_detector.model")

print("[INFO] starting video stream...")
vs = cv2.VideoCapture(0)

while True:
    ret, frame = vs.read()
    if not ret:
        break

    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

    for (box, pred) in zip(locs, preds):
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred

        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

        cv2.putText(frame, label, (startX, startY - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord("q"):
        break

vs.release()
cv2.destroyAllWindows()
